In [19]:
import pandas as pd
import torch
from torch import nn
from joblib import load


# Neural Network Model
class EsiModel(nn.Module):
    def __init__(self, input_size):
        super(EsiModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 5)  # 5 output classes for ESI 1-5
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


# Load the test data
test_data = pd.read_csv("test.csv")

# Ensure the test data has the same columns as the training data (excluding 'esi')
# You may need to adjust this based on your actual data
if "esi" in test_data.columns:
    test_data = test_data.drop(columns=["esi"])

# Handle missing values (you may want to adjust this based on your data)
test_data.fillna(0, inplace=True)

# Load the scaler
scaler = load("scaler.joblib")

# Preprocess the test data
test_data_scaled = scaler.transform(test_data)

# Load model
input_size = test_data.shape[1]
model = EsiModel(input_size)
model.load_state_dict(torch.load("esi_model.pth"))
model.eval()

# Convert to PyTorch tensor
test_tensor = torch.FloatTensor(test_data_scaled)

# Make predictions
with torch.no_grad():
    output = model(test_tensor)
    predicted_classes = torch.argmax(output, dim=1).numpy()

# Add 1 to predictions to match original ESI scale (1-5)
predicted_esi = predicted_classes + 1

# Add predictions to the test data
test_data["Predicted_ESI"] = predicted_esi

# Save results
test_data.to_csv("test_results.csv", index=False)

print("Predictions complete. Results saved to 'test_results.csv'.")

/var/folders/q8/w87pnqrn21b8x6m66ylf6d7h0000gn/T/ipykernel_6269/3204715402.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/Users/sgupta/WashUHackath

TypeError: 'collections.OrderedDict' object is not callable